# Hyperparameter Tuning using HyperDrive

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
from azureml.core import Workspace, Experiment, Environment, ScriptRunConfig
from azureml.core.compute import AmlCompute, ComputeTarget
from azureml.core.compute_target import ComputeTargetException
from azureml.core.model import InferenceConfig, Model
from azureml.core.webservice import AciWebservice
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.sampling import BayesianParameterSampling
from azureml.train.hyperdrive.parameter_expressions import uniform, choice
from azureml.widgets import RunDetails
import json
import requests

In [2]:
ws = Workspace.from_config()
experiment_name = 'PredictStrokeHyper'

experiment=Experiment(ws, experiment_name)

# Choose a name for your CPU cluster
cluster_name = "ClusterKal"

try:
    compute_target = ComputeTarget(workspace=ws, name= cluster_name)
    print("Found existing cluster,", cluster_name)
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='Standard_DS3_v2',
                                                           #vm_priority = 'lowpriority',
                                                           max_nodes=6)
    compute_target = ComputeTarget.create(ws, cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True, min_node_count = 0, timeout_in_minutes = 10)

Found existing cluster, ClusterKal
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


## Dataset

TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [3]:
%%writefile conda_dependencies.yml
dependencies:
- python=3.8
- scikit-learn
- pandas
- numpy
- pip:
  - azureml-defaults

Overwriting conda_dependencies.yml


## Hyperdrive Configuration

TODO: Explain the model you are using and the reason for chosing the different hyperparameters, termination policy and config settings.

In [4]:
# TODO: Create an early termination policy. This is not required if you are using Bayesian sampling.
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform, choice
from azureml.core.dataset import Dataset
from azureml.train.sklearn import SKLearn

early_termination_policy = BanditPolicy(evaluation_interval=2,slack_factor=0.2)

parameter_sampling = RandomParameterSampling(
                    {
                        "--C":uniform(0.05,0.10),
                        "--max_iter":choice(25,50,75,100) 
                    }
)

#for ML method 2
# Create a SKLearn estimator for use with train.py
from azureml.core import ScriptRunConfig
from azureml.core import Environment


#TODO: Create your estimator and hyperdrive config

sklearn_env = Environment.from_conda_specification('sklearn-env',"conda_dependencies.yml")

estimator = ScriptRunConfig(source_directory='.',
                            command=['python', 'train.py'],
                            compute_target=compute_target,
                            environment=sklearn_env)

hyperdrive_run_config = HyperDriveConfig(run_config=estimator,
                                         hyperparameter_sampling=parameter_sampling,
                                         primary_metric_name="AUC",
                                         primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                         max_total_runs=25,
                                         max_concurrent_runs=5)

In [5]:
#TODO: Submit your experiment

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [6]:
hyperdrive_run = experiment.submit(hyperdrive_run_config,show_output=True)

## Best Model

TODO: In the cell below, get the best model from the hyperdrive experiments and display all the properties of the model.

In [7]:
RunDetails(hyperdrive_run).show()
hyperdrive_run.wait_for_completion(show_output=True)

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

RunId: HD_57ae84b6-e997-431a-ace3-0364260d2f7b
Web View: https://ml.azure.com/runs/HD_57ae84b6-e997-431a-ace3-0364260d2f7b?wsid=/subscriptions/16bc73b5-82be-47f2-b5ab-f2373344794c/resourcegroups/kalakkad-ddl/workspaces/epe-venkat-aiml&tid=db05faca-c82a-4b9d-b9c5-0f64b6755421

Streaming azureml-logs/hyperdrive.txt

[2022-08-18T00:21:13.453686][GENERATOR][INFO]Trying to sample '5' jobs from the hyperparameter space
[2022-08-18T00:21:14.1011705Z][SCHEDULER][INFO]Scheduling job, id='HD_57ae84b6-e997-431a-ace3-0364260d2f7b_0' 
[2022-08-18T00:21:14.2253400Z][SCHEDULER][INFO]Scheduling job, id='HD_57ae84b6-e997-431a-ace3-0364260d2f7b_1' 
[2022-08-18T00:21:14.4397636Z][SCHEDULER][INFO]Successfully scheduled a job. Id='HD_57ae84b6-e997-431a-ace3-0364260d2f7b_0' 
[2022-08-18T00:21:14.3561149Z][SCHEDULER][INFO]Scheduling job, id='HD_57ae84b6-e997-431a-ace3-0364260d2f7b_2' 
[2022-08-18T00:21:14.4570656Z][SCHEDULER][INFO]Scheduling job, id='HD_57ae84b6-e997-431a-ace3-0364260d2f7b_3' 
[2022-08-18T00

{'runId': 'HD_57ae84b6-e997-431a-ace3-0364260d2f7b',
 'target': 'ClusterKal',
 'status': 'Completed',
 'startTimeUtc': '2022-08-18T00:21:12.566655Z',
 'endTimeUtc': '2022-08-18T00:33:14.845473Z',
 'services': {},
 'properties': {'primary_metric_config': '{"name":"AUC","goal":"maximize"}',
  'resume_from': 'null',
  'runTemplate': 'HyperDrive',
  'azureml.runsource': 'hyperdrive',
  'platform': 'AML',
  'ContentSnapshotId': 'bc693cda-a8b6-4466-8cf9-c37347440aa7',
  'user_agent': 'python/3.8.5 (Linux-5.4.0-1074-azure-x86_64-with-glibc2.10) msrest/0.6.21 Hyperdrive.Service/1.0.0 Hyperdrive.SDK/core.1.40.0',
  'space_size': 'infinite_space_size',
  'score': '0.7719341563786009',
  'best_child_run_id': 'HD_57ae84b6-e997-431a-ace3-0364260d2f7b_0',
  'best_metric_status': 'Succeeded',
  'best_data_container_id': 'dcid.HD_57ae84b6-e997-431a-ace3-0364260d2f7b_0'},
 'inputDatasets': [],
 'outputDatasets': [],
 'runDefinition': {'configuration': None,
  'attribution': None,
  'telemetryValues': {

## Model Deployment

Remember you have to deploy only one of the two models you trained but you still need to register both the models. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [8]:
best_run = hyperdrive_run.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()

TODO: In the cell below, send a request to the web service you deployed to test it.

In [10]:
#TODO: Save the best model
model = best_run.register_model(model_name='heart-failure-prediction-hyperdrive-model',model_path='outputs/hd-model.joblib', tags={'Method':'Hyperdrive'}, properties={'AUC': best_run_metrics['AUC']})
print(model)

Model(workspace=Workspace.create(name='epe-venkat-aiml', subscription_id='16bc73b5-82be-47f2-b5ab-f2373344794c', resource_group='kalakkad-ddl'), name=heart-failure-prediction-hyperdrive-model, id=heart-failure-prediction-hyperdrive-model:3, version=3, tags={'Method': 'Hyperdrive'}, properties={'AUC': '0.7719341563786009'})


TODO: In the cell below, print the logs of the web service and delete the service

**Submission Checklist**
- I have registered the model.
- I have deployed the model with the best accuracy as a webservice.
- I have tested the webservice by sending a request to the model endpoint.
- I have deleted the webservice and shutdown all the computes that I have used.
- I have taken a screenshot showing the model endpoint as active.
- The project includes a file containing the environment details.

